In [2]:
import utils
import numpy as np
import pandas as pd
import cv2
import h5py
import json
import typing

from pathlib import Path

In [5]:
data_folder = Path("data/hdf5v5_16bit")

with open('pre_process_cfg_local.json', 'r') as fd:
    pre_process_config = json.load(fd)

hdf5_path = pre_process_config["hdf5_path"]
dataframe_path = pre_process_config["dataframe_path"]
stations = pre_process_config["stations"]
target_channels = ["ch1","ch2","ch3","ch4","ch6"]
utils.viz_hdf5_imagery(hdf5_path, target_channels, dataframe_path, stations)

In [6]:
def keys(f):
    return [key for key in f.keys()]

In [9]:
hdf5_offset = 33  # this would correspond to 2015.01.01.0800 + (32)*15min = 2015.01.01.1600
with h5py.File(hdf5_path, 'r') as h5_data:
    ch1_data = utils.fetch_hdf5_sample("ch1", h5_data, hdf5_offset)
    print("ch1_data:",ch1_data.shape)
    print(*list(h5_data.values()),sep = "\n")
    GHI = h5_data["BND_GHI"][()]
    ch1 = h5_data["ch1"][()]
    ch1_lut = h5_data["ch1_LUT"][()]
    lat = h5_data["lat"][()]
    lon = h5_data["lon"][()]
    ch1_0 = utils.fetch_hdf5_sample("ch1", h5_data, 0)
    lat_0 = utils.fetch_hdf5_sample("lat", h5_data, 0)
    print("==========")
    print(*list(h5_data["ch1"].attrs),sep = "\n")
    print("values:")
    print(h5_data["ch1"].attrs["compr_args"])
    print(h5_data["ch1"].attrs["compr_type"])
    print(h5_data["ch1"].attrs["force_cvt_uint16"])
    print(h5_data["ch1"].attrs["force_cvt_uint8"])
    print(h5_data["ch1"].attrs["orig_dtype"])
    print(h5_data["ch1"].attrs["orig_max"])
    print(h5_data["ch1"].attrs["orig_min"])
    print(h5_data["ch1"].attrs["orig_shape"])
    print("==========")
    print(*list(h5_data["lat"].attrs),sep = "\n")
    print("values:")
    print(h5_data["lat"].attrs["force_cvt_uint16"])
    print(h5_data["lat"].attrs["force_cvt_uint8"])
    print(h5_data["lat"].attrs["orig_dtype"])
    print(h5_data["lat"].attrs["orig_shape"])
    #lat = h5_data["lat"][()]

ch1_data: (650, 1500)
<HDF5 dataset "BND_GHI": shape (96,), type "<f8">
<HDF5 dataset "BND_GHI_LUT": shape (96,), type "<i4">
<HDF5 dataset "DRA_GHI": shape (96,), type "<f8">
<HDF5 dataset "DRA_GHI_LUT": shape (96,), type "<i4">
<HDF5 dataset "FPK_GHI": shape (96,), type "<f8">
<HDF5 dataset "FPK_GHI_LUT": shape (96,), type "<i4">
<HDF5 dataset "GWN_GHI": shape (96,), type "<f8">
<HDF5 dataset "GWN_GHI_LUT": shape (96,), type "<i4">
<HDF5 dataset "PSU_GHI": shape (96,), type "<f8">
<HDF5 dataset "PSU_GHI_LUT": shape (96,), type "<i4">
<HDF5 dataset "SXF_GHI": shape (96,), type "<f8">
<HDF5 dataset "SXF_GHI_LUT": shape (96,), type "<i4">
<HDF5 dataset "TBL_GHI": shape (96,), type "<f8">
<HDF5 dataset "TBL_GHI_LUT": shape (96,), type "<i4">
<HDF5 dataset "ch1": shape (86,), type "|O">
<HDF5 dataset "ch1_LUT": shape (96,), type "<i4">
<HDF5 dataset "ch2": shape (86,), type "|O">
<HDF5 dataset "ch2_LUT": shape (96,), type "<i4">
<HDF5 dataset "ch3": shape (86,), type "|O">
<HDF5 dataset "

In [10]:
ch1_data2 = np.around(ch1_data*255/(ch1_data.max()-ch1_data.min()))
print(ch1_data2.min())
print(ch1_data2.max())

-0.0
255.0


In [13]:
cv2.imwrite("ch1.png",ch1_data2)

True

In [14]:
with h5py.File(hdf5_path, 'r') as h5_data:
    lats, lons = utils.fetch_hdf5_sample("lat", h5_data, hdf5_offset),\
    utils.fetch_hdf5_sample("lon", h5_data, hdf5_offset)

In [15]:
print(stations.items())

dict_items([('BND', [40.05192, -88.37309, 230]), ('TBL', [40.12498, -105.2368, 1689]), ('DRA', [36.62373, -116.01947, 1007]), ('FPK', [48.30783, -105.1017, 634]), ('GWN', [34.2547, -89.8729, 98]), ('PSU', [40.72012, -77.93085, 376]), ('SXF', [43.73403, -96.62328, 473])])


In [16]:
window_size = 20
img = cv2.imread("ch1.png")
cv2.rectangle(img, (915-window_size, 401-window_size), (915+window_size, 401+window_size), (0,0,255), 2)
cv2.putText(img,"BND",(915-15,401-30),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1)

cv2.rectangle(img, (878-window_size, 256-window_size), (878+window_size, 256+window_size), (0,255,0), 2)
cv2.putText(img,"GWN",(878-15,256-30),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)

cv2.rectangle(img, (494-window_size, 403-window_size), (494+window_size, 403+window_size), (0,150,150), 2)
cv2.putText(img,"TBL",(494-15,403-30),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,150,150),1)

cv2.rectangle(img, (224-window_size, 315-window_size), (224+window_size, 315+window_size), (0,255,255), 2)
cv2.putText(img,"DRA",(224-15,315-30),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,255),1)

cv2.rectangle(img, (497-window_size, 607-window_size), (497+window_size, 607+window_size), (255,255,0), 2)
cv2.putText(img,"FPK",(497-15,607-30),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,0),1)

cv2.rectangle(img, (1176-window_size, 418-window_size), (1176+window_size, 418+window_size), (255,0,255), 2)
cv2.putText(img,"PSU",(1176-15,418-30),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,0,255),1)

cv2.rectangle(img, (709-window_size, 493-window_size), (709+window_size, 493+window_size), (0,150,250), 2)
cv2.putText(img,"SXF",(709-15,493-30),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,150,250),1)

img = cv2.flip(img, 0)


cv2.imwrite("US_ch1.png",img)
cv2.imshow("US_ch1", img)
